In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import chardet
#df = pd.read_csv("sales_total_sku_channel.csv",low_memory=False)
%matplotlib inline

## 货品维度

#### 考虑该货品的品类，颜色，推出至今的年份（考虑到款式可能渐渐过时等），标牌价格，售卖价格，折扣率，地区（气候与消费能力），材料，系列等
#### 对销售量，尺码比例等进行预测


In [2]:
goods = pd.read_csv("goods.csv")

/home/gang/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
goods.columns

Index(['sale_date', 'category_sub', 'year', 'color_name', 'sale_price',
       'tag_price', 'season', 'series', 'province', 'material', 'size_name',
       'Quantity', 'discount', 'release_years', 'month', '季节', 'sku_id',
       '周数-年', 'goods_no', 'category'],
      dtype='object')

 周数-年是指销售日期是这一年的第几周，release_years是销售的年份减去发行的年份，seaon是原数据提供的，应该是指品类对应的季节，“季节”是销售日期的季节

'release_years' 是销售年份减去发行年份
discount是 销售价格除以标牌价格

## 渠道维度

#### 除了考虑前面的属性，还要考虑门店的面积，导购员数量，地址（可以归总地址是否热门）
#### 对销售量，尺码比例等进行预测

In [4]:
df_channel = pd.read_csv("goods_channel.csv")

/home/gang/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (19,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df_channel.columns

Index(['sale_date', 'category_sub', 'year', 'color_name', 'sale_price',
       'tag_price', 'season', 'series', 'province', 'area',
       'shopping_guide_quantity', 'address', 'channel_type', 'channel_name',
       'material', 'size_name', 'Quantity', 'release_years', '季节', 'sku_id',
       '周数-年', 'goods_no', 'category'],
      dtype='object')

In [6]:
group = df_channel.groupby(by = "channel_name")

In [7]:
for key,item in group:
    group.get_group(key).to_csv("店铺/"+key.replace("/","_")+".csv",index = False)
    

按照店铺分别输出了csv文件

In [8]:
group_year = df_channel.groupby(by = "year")

In [9]:
for key,item in group_year:
    group_year.get_group(key).to_csv("年份/"+str(key)+".csv",index = False)

按照年份分别输出了csv文件

In [10]:
df_channel['channel_name'].head(5)

0    重庆王府井百货
1     深圳太阳百货
2      南京水游城
3     深圳太阳百货
4    重庆王府井百货
Name: channel_name, dtype: object

可以根据同一年份同一地区的不同地址销售额度来看地址是否有影响， 这边的销售额可以定义为 总销售额/面积 等，进行规整

### 订单的归总

In [11]:
purchase = pd.read_csv("New_data/purchase_20190410104706.csv")

In [12]:
df_sku = goods[['sku_id','season','series','goods_no','category','category_sub','color_name','tag_price','material', 'size_name']].groupby(by = 'sku_id').head(1)

In [13]:
df_sku.head(5)

,sku_id,season,series,goods_no,category,category_sub,color_name,tag_price,material,size_name
0,121802023B701#B701#00#L,春季,Modern City,121802023B701,夹克,夹克,黑色,699.0,皇家细格尼丝纺,L
97,112804029B365#B365#00#M,夏季,Other,112804029B365,T恤,T恤,薄荷绿,199.0,NaN,M
167,121904087B902#B902#00#M,春季,Modern City,121904087B902,T恤,T恤,麻灰,229.0,32'S 单纱汗布,M
248,1218020032B01#2B01#00#M,春季,Denim,1218020032B01,夹克,夹克,蓝牛,799.0,美术牛仔,M
530,1219070042B01#2B01#32#29,春季,Denim,1219070042B01,牛仔长裤,牛仔长裤,蓝牛,599.0,弹力黛林牛仔,29


In [14]:
new_df = pd.merge(purchase,df_sku,on="sku_id")

In [15]:
new_df.head(5)

,purchase_no,purchase_date,supplier_id,sku_id,purchase_price,Quantity,season,series,goods_no,category,category_sub,color_name,tag_price,material,size_name
0,POB0000002,2013/06/03,GYS074,133894008609#609#00#L,61.0,121,秋季,Modern Vintage,133894008609,皮带,配件,藏黑蓝,349.0,牛皮,L
1,POB0000002,2013/06/03,GYS074,133894009402#402#00#M,79.0,93,秋季,Modern Vintage,133894009402,皮带,配件,咖啡棕,349.0,牛皮,M
2,POB0000002,2013/06/03,GYS074,133994002104#104#00#M,70.0,114,秋季,Modern Vintage,133994002104,皮带,配件,果酱红,299.0,牛皮,M
3,POB0000002,2013/06/03,GYS074,133994002104#104#00#S,70.0,87,秋季,Modern Vintage,133994002104,皮带,配件,果酱红,299.0,牛皮,S
4,POB0000002,2013/06/03,GYS074,134894001402#402#00#L,70.0,116,冬季,Modern Vintage,134894001402,皮带,配件,咖啡棕,349.0,牛皮,L


自此可以汇总订单量以及利用货品的属性预测订单量，或者统计订单量的尺寸比例等.也可以根据季节等。 这里的season是货品对应季节，再对应采购日期的季节或许会有帮助

In [16]:
def season(i):
    if i in [3,4,5]:
        return "春"
    if i in [6,7,8]:
        return "夏"
    if i in [9,10,11]:
        return "秋"
    if i in [12,1,2]:
        return "冬"
    return np.nan

In [17]:
new_df["purchase_month"] = pd.to_datetime(new_df["purchase_date"]).apply(lambda x : x.strftime("%m")).astype(int)
new_df["采购季节"]=new_df["purchase_month"].apply(season)

In [18]:
new_df["purchase_year"] = pd.to_datetime(new_df["purchase_date"]).apply(lambda x : x.strftime("%Y")).astype(int)

In [136]:
new_df.head(5)

,purchase_no,purchase_date,supplier_id,sku_id,purchase_price,Quantity,season,series,goods_no,小类,category_sub,color_name,tag_price,material,size_name,purchase_month,采购季节,purchase_year,品类
0,POB0000002,2013/06/03,GYS074,133894008609#609#00#L,61.0,121,秋季,Modern Vintage,133894008609,皮带,配件,藏黑蓝,349.0,牛皮,L,6,夏,2013,皮带
1,POB0000002,2013/06/03,GYS074,133894009402#402#00#M,79.0,93,秋季,Modern Vintage,133894009402,皮带,配件,咖啡棕,349.0,牛皮,M,6,夏,2013,皮带
2,POB0000002,2013/06/03,GYS074,133994002104#104#00#M,70.0,114,秋季,Modern Vintage,133994002104,皮带,配件,果酱红,299.0,牛皮,M,6,夏,2013,皮带
3,POB0000002,2013/06/03,GYS074,133994002104#104#00#S,70.0,87,秋季,Modern Vintage,133994002104,皮带,配件,果酱红,299.0,牛皮,S,6,夏,2013,皮带
4,POB0000002,2013/06/03,GYS074,134894001402#402#00#L,70.0,116,冬季,Modern Vintage,134894001402,皮带,配件,咖啡棕,349.0,牛皮,L,6,夏,2013,皮带


In [137]:
group_year = new_df.groupby(by = "purchase_year")

In [138]:
for key,item in group_year:
    group_year.get_group(key).to_csv("年份/Purchase_"+str(key)+".csv",index = False)

In [139]:
new_df.to_csv("purchase_info.csv",index = False)

选出2013年的来做一下

In [25]:
new_df["purchase_year"].value_counts()

2017    13212
2014     9480
2016     6140
2015     5686
2018     4643
2013     4571
Name: purchase_year, dtype: int64

In [43]:
purchase_2013 = new_df[new_df["purchase_year"]==2013]

In [140]:
purchase_2013 = pd.read_csv("年份/Purchase_2013.csv")

这里的折扣率就用2013年全年的折扣率（销售额/（销售数量×吊牌价））吧，虽然这样不严谨。

In [141]:
df_2013 = pd.read_csv("年份/2013.csv")

In [142]:
df_2013.columns

Index(['sale_date', 'category_sub', 'year', 'color_name', 'sale_price',
       'tag_price', 'season', 'series', 'province', 'area',
       'shopping_guide_quantity', 'address', 'channel_type', 'channel_name',
       'material', 'size_name', 'Quantity', 'release_years', '季节', 'sku_id',
       '周数-年', 'goods_no', 'category'],
      dtype='object')

In [143]:
df_2013["sales"] = df_2013["sale_price"]*df_2013["Quantity"]

In [144]:
df_2013["sales_without_discount"]=df_2013["tag_price"]*df_2013["Quantity"]#如果没有折扣，会是多少钱，用来统计折扣率

### 这里的销售额以及原价情况下的销售额也是一整年的

In [145]:
group_2013 = df_2013[['sku_id','sales','sales_without_discount','Quantity']].groupby(by = "sku_id")

In [146]:
sales_2013=group_2013.sum()

In [147]:
sales_2013.head(3)

,sales,sales_without_discount,Quantity
sku_id,,,
0718010020314#0314#30#28,869.0,1198.0,2.0
0718010020314#0314#30#32,307.0,349.0,1.0
0718020010322#0322#00#XL,1438.0,1798.0,2.0


In [148]:
sales_2013.reset_index(inplace = True)

In [149]:
sales_2013.head(3)

,sku_id,sales,sales_without_discount,Quantity
0,0718010020314#0314#30#28,869.0,1198.0,2.0
1,0718010020314#0314#30#32,307.0,349.0,1.0
2,0718020010322#0322#00#XL,1438.0,1798.0,2.0


In [150]:
sales_2013["折扣率"]=sales_2013["sales"]/sales_2013["sales_without_discount"]

In [151]:
sales_2013.head(3)

,sku_id,sales,sales_without_discount,Quantity,折扣率
0,0718010020314#0314#30#28,869.0,1198.0,2.0,0.725376
1,0718010020314#0314#30#32,307.0,349.0,1.0,0.879656
2,0718020010322#0322#00#XL,1438.0,1798.0,2.0,0.799778


In [152]:
purchase_2013_ = pd.merge(purchase_2013,sales_2013,on="sku_id")

In [153]:
purchase_2013_.head(3) 

,purchase_no,purchase_date,supplier_id,sku_id,purchase_price,Quantity_x,season,series,goods_no,小类,...,material,size_name,purchase_month,采购季节,purchase_year,品类,sales,sales_without_discount,Quantity_y,折扣率
0,POB0000002,2013/06/03,GYS074,134894001402#402#00#L,70.00,116,冬季,Modern Vintage,134894001402,皮带,...,牛皮,L,6,夏,2013,皮带,19238.5,26810.0,40.0,0.717587
1,POB0000004,2013/06/03,GYS079,133798003611#611#00#38,184.15,93,秋季,Modern Vintage,133798003611,鞋,...,巴布牛仔,38,6,夏,2013,鞋,17340.5,23370.0,30.0,0.741998
2,POB0000004,2013/06/03,GYS079,133798003611#611#00#42,184.15,149,秋季,Modern Vintage,133798003611,鞋,...,巴布牛仔,42,6,夏,2013,鞋,18108.0,35658.0,42.0,0.507824


In [58]:
((purchase_2013_["Quantity_x"]/purchase_2013_["Quantity_y"])<1).value_counts()

False    338
True      39
dtype: int64

In [154]:
purchase_2013_.rename(index=str, columns={"Quantity_x": "Quantity_purchase", "Quantity_y": "Quantity_sales"},inplace = True)

In [155]:
purchase_2013_.columns

Index(['purchase_no', 'purchase_date', 'supplier_id', 'sku_id',
       'purchase_price', 'Quantity_purchase', 'season', 'series', 'goods_no',
       '小类', 'category_sub', 'color_name', 'tag_price', 'material',
       'size_name', 'purchase_month', '采购季节', 'purchase_year', '品类', 'sales',
       'sales_without_discount', 'Quantity_sales', '折扣率'],
      dtype='object')

In [156]:
purchase_2013.to_csv("purchase_2013.csv", index =False)

In [166]:
stat = purchase_2013_[["goods_no","品类","小类","Quantity_purchase"]]

In [167]:
stats = stat.groupby(by = ["goods_no","品类","小类"]).sum()

然后可以用groupby来计算每个category_sub下面的sku的数量，分别的价格，折扣率，全年的销售额等等

In [168]:
stats.head(3)

,,,Quantity_purchase
goods_no,品类,小类,
121804086B701,T恤,T恤,100
121804086B801,T恤,T恤,33
133798003611,鞋,鞋,242


In [169]:
stats.reset_index(inplace = True)

In [170]:
stats.shape

(206, 4)

In [171]:
stats.head(3)

,goods_no,品类,小类,Quantity_purchase
0,121804086B701,T恤,T恤,100
1,121804086B801,T恤,T恤,33
2,133798003611,鞋,鞋,242


In [181]:
info = purchase_2013_[["goods_no",'折扣率','tag_price','purchase_price',"Quantity_sales"]]

In [182]:
info = info.groupby(by = "goods_no").head(1)

In [185]:
stats2013=pd.merge(info,stats, on = "goods_no")

In [188]:
stats2013.columns

Index(['goods_no', '折扣率', 'tag_price', 'purchase_price', 'Quantity_sales',
       '品类', '小类', 'Quantity_purchase'],
      dtype='object')

In [ ]:
x = stats2013[]

In [189]:
stats2013.to_csv("stats2013.csv",index = False)

生成了2013年用来预测的数据，用同样方法统计其他年份的，之后用lstm，arima，或者用xgboost（把往年的销量，yi也当做属性）

## 用来实现drive上面的分类

In [106]:
Class = dict()

In [120]:
import math
for rows in range(19):
    for columns in range(14):
        if(type(Cls.iloc[rows,columns])==str):
            Class[Cls.iloc[rows,columns]]=Cls.iloc[rows,0]

In [121]:
Class

{'夹克': '夹克',
 '牛仔夹克': '夹克',
 '针织夹克': '夹克',
 '单夹克': '夹克',
 '皮夹克': '夹克',
 '衬衫': '衬衫',
 '夹棉衬衫': '衬衫',
 '短袖衬衫': '衬衫',
 '牛仔衬衫': '衬衫',
 '长袖牛仔衬衫': '衬衫',
 '长袖衬衫': '衬衫',
 '短袖牛仔衬衫': '衬衫',
 '长袖牛仔夹克式衬衫': '衬衫',
 '长袖夹克式衬衫': '衬衫',
 '羽绒衬衫': '衬衫',
 '短袖牛仔夹克式衬衫': '衬衫',
 '短袖夹克式衬衫': '衬衫',
 '长裤': '长裤',
 '梭织长裤': '长裤',
 '针织长裤': '长裤',
 '牛仔长裤': '长裤',
 '牛仔运动裤': '长裤',
 '非牛仔运动裤': '长裤',
 'T恤': 'T恤',
 '短袖T恤': 'T恤',
 '长袖T恤': 'T恤',
 '短袖TEE': 'T恤',
 'POLO': 'T恤',
 '短袖Polo': 'T恤',
 '卫衣': '卫衣',
 '毛衣': '毛衣',
 '毛衣开衫': '毛衣',
 '裙': '裙',
 '牛仔裙': '裙',
 '针织连衣裙': '裙',
 '半裙': '裙',
 '牛仔半裙': '裙',
 '连身裙': '裙',
 '连身': '裙',
 '牛仔连身': '裙',
 '短裤': '短裤',
 '牛仔短裤': '短裤',
 '梭织短裤': '短裤',
 '短牛仔裤': '短裤',
 '棉服': '棉服',
 '短棉服': '棉服',
 '中长棉服': '棉服',
 '长棉服': '棉服',
 '羽绒服': '羽绒服',
 '长羽绒服': '羽绒服',
 '短羽绒服': '羽绒服',
 '中长羽绒服': '羽绒服',
 '短款羽绒服': '羽绒服',
 '超轻羽绒服': '羽绒服',
 '羽绒背心': '羽绒服',
 '西装': '西装',
 '背心': '背心',
 '牛仔马甲': '背心',
 '马甲': '背心',
 '牛仔背心': '背心',
 '棉背心': '背心',
 '大衣': '大衣',
 '风衣外套': '大衣',
 '毛呢': '大衣',
 '风衣': '大衣',
 '皮带': '皮带',
 '腰带': '皮带',
 '包': '包',
 '

In [122]:
cls=new_df['category'].map(Class)

In [134]:
new_df.rename(index=str, columns={"category":"小类"},inplace = True)

In [128]:
new_df["品类"] = cls

In [135]:
new_df.columns

Index(['purchase_no', 'purchase_date', 'supplier_id', 'sku_id',
       'purchase_price', 'Quantity', 'season', 'series', 'goods_no', '小类',
       'category_sub', 'color_name', 'tag_price', 'material', 'size_name',
       'purchase_month', '采购季节', 'purchase_year', '品类'],
      dtype='object')

In [125]:
cls.value_counts()

长裤     17094
T恤      7064
夹克      4343
衬衫      3547
短裤      2665
卫衣      2100
裙       1982
羽绒服     1724
棉服      1213
大衣       558
鞋        428
皮带       268
背心       255
配件       115
毛衣       107
包         94
西装        62
杂物         4
Name: category, dtype: int64